In [1]:
import pandas as pd
import geopandas as gpd
import scipy as sp
from scipy.spatial import KDTree
import numpy as np
import networkx as nx
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.metrics import mean_squared_log_error
%matplotlib inline
ox.config(use_cache=True, log_console=True)
ox.__version__

'0.11'

In [ ]:
def get_route(row, G):
    """Pandas apply function for getting the best route coordinates between two coordinates"""
    try:
        # get the nearest network nodes to two points
        orig = ox.get_nearest_node(G, (row.pickup_latitude, row.pickup_longitude))
        dest = ox.get_nearest_node(G, (row.dropoff_latitude, row.dropoff_longitude))

        # find the route between these nodes then plot it
        route = nx.shortest_path(G, orig, dest, weight='travel_time')
        res = []
        for node in route:
            res.append((G.node[node]['y'], G.node[node]['x']))
    except Exception as e:
        print(e)
        res = []
    row['route'] = res
    return row

In [ ]:
def calc_distance(cord1, cord2):
    """Calculate distance in meters between two coordinates"""
    # approximate radius of earth in km
    R = 6373.0

    lat1 = np.radians(cord1[0])
    lon1 = np.radians(cord1[1])
    lat2 = np.radians(cord2[0])
    lon2 = np.radians(cord2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c * 1000
    return distance

calc_distance((40.735018, -74.003515), (40.735018, -74.003215))

In [ ]:
def get_graph(query):
    """Get a networkx osmnx graph (nodes and edges) of a location defined by query"""
    # get a graph for some city
    G = ox.graph_from_place('Manhattan, New York City, New York, USA', network_type='drive')
    fig, ax = ox.plot_graph(G, node_zorder=2, node_color='w', bgcolor='k')
    
    return G

In [2]:
def plot_route(G, orig, dest):
    """Plot car route between two points in a graph from osmnx"""
    # get the nearest network nodes to two points
    orig = ox.get_nearest_node(G, orig)
    dest = ox.get_nearest_node(G, dest)

    # find the route between these nodes then plot it
    route = nx.shortest_path(G, orig, dest, weight='travel_time')
    fig, ax = ox.plot_graph_route(G, route, route_linewidth=6, node_size=0, bgcolor='k')

In [ ]:
def evaluate(df, col1, col2):
    """Calculate root mean squared log error"""
    df = df.drop_duplicates(subset='id').copy()
    return np.sqrt(mean_squared_log_error(test_UNIQUE.trip_duration, test_UNIQUE.pred))